In [3]:
### header ###
__author__ = "Jenhan Tao"
__license__ = "MIT"
__email__ = "jenhantao@gmail.com"

### imports ###
import sys # system functions
import os # os functions
import pandas as pd # for reading data
import numpy as np # for numeric operations
import matplotlib.pyplot as plt 
import matplotlib # for visualizing data
import scipy # scientific computing
import seaborn as sns # for pretty plots

### notebook specific configuration ###
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 200
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# it's bad practice to work within a github repo and also bad practice to have data in a github repo
# but it's convenient
# change working directory to github repo
working_directory = '/gpfs/data01/glasslab/home/jtao/side_projects/fantasy_football_genetic_algorithm_draft/'
os.chdir(working_directory)

### Read in rankings

In [17]:
# year_rankFrame_dict = {} # {year:DataFrame}
# for f in os.listdir(working_directory + '/rankings_ffc_standard'):
#     json_path = working_directory + '/rankings_ffc_standard/' + f
#     ranks = pd.read_json(json_path, typ='series')
#     adp_list = [x['adp'] for x in ranks['players']]
#     name_list = [x['name'] for x in ranks['players']]
#     position_list = [x['position'] for x in ranks['players']]
    
#     year = f.split('_')[-1].replace('.json','')
    
#     rankFrame = pd.DataFrame({'ADP':adp_list, 'Position':position_list, 'Name':name_list})
#     year_rankFrame_dict[year] = rankFrame
    


In [168]:
year_rankFrame_dict = {} # {year:DataFrame}
for f in os.listdir(working_directory + '/rankings_mfl_standard'):
    csv_path = working_directory + '/rankings_mfl_standard/' + f
    rankFrame = pd.read_csv(csv_path, sep=',')
    rankFrame['POSITION'] = [x.split(' ')[-1].replace('*','') for x in rankFrame['PLAYER'].values]
    rankFrame['NAME'] = [' '.join(x.split(', ')[1].split(' ')[:-2]) + ' ' + x.split(', ')[0].replace(',','') for x in rankFrame['PLAYER'].values]
    
    rankFrame = rankFrame[rankFrame['POSITION'].isin(['Def', 'PK', 'QB', 'RB', 'TE', 'WR'])]
    rankFrame['ADP SCORE'] = rankFrame['AVG. PICK'].max() - rankFrame['AVG. PICK'].values
    year = f.split('_')[-1].replace('.csv','')
    
    year_rankFrame_dict[year] = rankFrame
    


### Read in season results

In [123]:
year_scoreFrame_dict = {} # {year:DataFrame}
for f in os.listdir(working_directory + '/scoring_standard'):
    score_path = working_directory + '/scoring_standard/' + f
    scoreFrame = pd.read_csv(score_path, sep=',')
    year = f.split('_')[-1].replace('.csv','')
    year_scoreFrame_dict[year] = scoreFrame
    


In [124]:
scoreFrame.head()

,Player,Points
0,Stephen Gostkowski,159
1,Graham Gano,150
2,Blair Walsh,147
3,Robbie Gould,141
4,Josh Brown,140


### Create Players

In [195]:
position_index_dict = dict(zip(sorted(rankFrame['POSITION'].unique()), 
                               range(len(rankFrame['POSITION'].unique()))))
index_position_dict = dict(zip(range(len(rankFrame['POSITION'].unique())),
                                sorted(rankFrame['POSITION'].unique()), 
                               ))

In [196]:
position_index_dict # gives the position and the corresponding index

{'Def': 0, 'PK': 1, 'QB': 2, 'RB': 3, 'TE': 4, 'WR': 5}

In [197]:
index_position_dict # gives the index and the corresponding index

{0: 'Def', 1: 'PK', 2: 'QB', 3: 'RB', 4: 'TE', 5: 'WR'}

In [128]:
def create_new_players(num_players, num_rounds):
    '''
    inputs: number of players and rounds
    outputs: list of fantasy players
    '''
    num_positions = 6 # number of positions
    player_list = []
    for player_number in range(num_players):
        new_player = []
        for round_number in range(num_rounds):
            weights = np.random.random(num_positions) # initialize random weights
            normed_weights = weights/np.sum(weights) # normalize so weights sum to 1
            new_player.append(normed_weights)
    
        player_list.append((new_player))
    return np.array(player_list)

In [151]:
len(rankFrame['POSITION'].unique()) # number of positions

9

In [40]:
fantasy_players = create_new_players(12, 10)

In [41]:
fantasy_players.shape # first index gives the player
                      # second index gives the round
                      # third index gives the position 


(12, 10, 6)

In [ ]:
fantasy_players[0,0,0] #gives the importance for player 1 in the first round for a defense 
fantasy_players[0,0,1] #gives the importance for player 1 in the first round for a place kicker (PK)
fantasy_players[8,3,2] #gives the importance for player 9 in the fourth round for a QB

In [53]:
fantasy_players.shape

(12, 10, 6)

### Run Simulation

In [179]:
rankFrame = year_rankFrame_dict['2017']

scoreFrame = year_scoreFrame_dict['2017']

In [180]:
rankFrame.shape

(340, 10)

In [181]:
scoreFrame.shape

(461, 2)

In [182]:
score_players = set (scoreFrame['Player'].values)

In [183]:
rank_players = set(rankFrame['NAME'].values)

In [184]:
len(score_players - rank_players)

186

In [185]:
len(rank_players-score_players)

65

In [171]:
rankFrame.head()

,#,PICK,PLAYER,AVG. PICK,MIN. PICK,MAX. PICK,# DRAFTS SELECTED IN,POSITION,NAME,ADP SCORE
0,1,1.01,"Foster, Arian HOU RB",2.35,1,110,954,RB,Arian Foster,202.58
1,2,1.02,"Rodgers, Aaron GBP QB",2.65,1,109,930,QB,Aaron Rodgers,202.28
2,3,1.03,"Rice, Ray BAL RB",3.92,1,111,954,RB,Ray Rice,201.01
3,4,1.04,"McCoy, LeSean PHI RB",4.84,1,113,953,RB,LeSean McCoy,200.09
4,5,1.05,"Brady, Tom NEP QB",5.62,1,117,930,QB,Tom Brady,199.31


In [172]:
scoreFrame.head()

,Player,Unnamed: 1
0,Russell Wilson,402
1,Tom Brady,352
2,Carson Wentz,338
3,Alex Smith,334
4,Cam Newton,331


In [226]:
fantasy_players.shape

(12, 10, 6)

In [227]:
def simulate_draft(rankFrame, fantasy_players, num_rounds):
    '''
    returns a list of rosters for each fantasy player
    '''
    team_rosters = [[] for x in range(fantasy_players.shape[0])]
    
    for player in 
    
    drafted_players = set()
    for draft_round in range(num_rounds):
        for player in range(fantasy_players.shape[0]):
            # retrieve strategy
            round_strategy = fantasy_players[player,draft_round, :]
            player_score_tuples = []
            # calculate player ranks
            for index, row in rankFrame[~rankFrame['NAME'].isin(drafted_players)].iterrows(): 
                position = row['POSITION']
                position_weight = round_strategy[position_index_dict[position]]
                weighted_player_score = position_weight * float(row['ADP SCORE'])
                player_score_tuples.append((row['NAME'], weighted_player_score))
            
            # draft player
            player_score_tuples.sort(key = lambda x: x[1], reverse=True)
            drafted_player = player_score_tuples[0][0]
            
            team_rosters[player].append(drafted_player)
            drafted_players.add(drafted_player)
    return team_rosters

In [228]:
rosters = simulate_draft(rankFrame, fantasy_players, 10)

In [231]:
#  check rosters don't overlap
for i in range(len(rosters)-1):
    for j in range(i+1, len(rosters)):
        team1 = rosters[i]
        team2 = rosters[j]
        print(i, j, len(set(team1).intersection(set(team2))))

0 1 0
0 2 0
0 3 0
0 4 0
0 5 0
0 6 0
0 7 0
0 8 0
0 9 0
0 10 0
0 11 0
1 2 0
1 3 0
1 4 0
1 5 0
1 6 0
1 7 0
1 8 0
1 9 0
1 10 0
1 11 0
2 3 0
2 4 0
2 5 0
2 6 0
2 7 0
2 8 0
2 9 0
2 10 0
2 11 0
3 4 0
3 5 0
3 6 0
3 7 0
3 8 0
3 9 0
3 10 0
3 11 0
4 5 0
4 6 0
4 7 0
4 8 0
4 9 0
4 10 0
4 11 0
5 6 0
5 7 0
5 8 0
5 9 0
5 10 0
5 11 0
6 7 0
6 8 0
6 9 0
6 10 0
6 11 0
7 8 0
7 9 0
7 10 0
7 11 0
8 9 0
8 10 0
8 11 0
9 10 0
9 11 0
10 11 0


In [209]:
round_strategy = fantasy_players[0,0, :]

In [210]:
round_strategy

array([0.12450951, 0.24221658, 0.22053774, 0.201553  , 0.10757643,
       0.10360674])

In [199]:
position_index_dict

{'Def': 0, 'PK': 1, 'QB': 2, 'RB': 3, 'TE': 4, 'WR': 5}

In [204]:
for index, row in rankFrame[~rankFrame['NAME'].isin(set())].iterrows():
    break

In [211]:
round_strategy[position_index_dict[row['POSITION']]]

0.20155300215822272

In [ ]:
def score_roster(score_frame, fantasy_teams):
    '''
    returns a list of performances for each fantasy player's team
    '''
    
    return fantasy_team_performances